# Etude des motifs - Antoine

Nous étudions les motifs présents dans le corpus d'Antoine. 
1. Dans un premier temps on se base sur les retranscriptions phonétiques d'Antoine issu de l'alphabet phonétique. 
2. Puis on se base les retranscriptions phonémiques issue de l'alphabet Fançais. 
3. Finalement, on se base sur les retranscriptions phonétiques d'Antoine que l'on pondèrent par la localisation du phonème dans le mot et par sa "valeur" de prononciation.  


Dans cette étude, on définit les items comme :
 - Un mot
 - Un phonème
 
Pour chaque items on utilise des algorithmes : 
 - Frequent Itemset Mining
 - Sequential Pattern Extraction
 
 Le but de cette étude est d'extraire les motifs fréquents du corpus.


In [1]:
from pymining import itemmining, assocrules, seqmining
import pandas as pd
import os

## Chargement des données

In [2]:
os.chdir('C:\\Users\\Lamou\\Desktop\\MIASHS\\TER_a\\data')
data_antoine = pd.read_csv('data_antoine_final.csv',
                           sep = '\t',
                           encoding = 'utf-8',
                           index_col=False)

# On supprime la colonne qui duplique les index
data_antoine = data_antoine.drop(columns = 'Unnamed: 0')

On récupère les retranscritpions phonétiques et phonémiques

In [3]:
chi_antoine = data_antoine[data_antoine['type'] == 'CHI']
pho_antoine = data_antoine[data_antoine['type'] == 'pho']

In [4]:
pho_antoine.head()

,enfant,age,type,seconde_debut,seconde_fin,contenu
1228,ANTOINE,1_00_24,pho,NaN,NaN,a
1230,ANTOINE,1_00_24,pho,NaN,NaN,a
1319,ANTOINE,1_00_24,pho,NaN,NaN,ahaa
1326,ANTOINE,1_00_24,pho,NaN,NaN,a
1342,ANTOINE,1_00_24,pho,NaN,NaN,ajabə


In [5]:
#chi_antoine.loc[chi_antoine['contenu']=='yyy .'] # Contient les phrases phonémiques 'yyy .'

In [6]:
#pho_antoine = pho_antoine['contenu']
pho_antoine.head()

,enfant,age,type,seconde_debut,seconde_fin,contenu
1228,ANTOINE,1_00_24,pho,NaN,NaN,a
1230,ANTOINE,1_00_24,pho,NaN,NaN,a
1319,ANTOINE,1_00_24,pho,NaN,NaN,ahaa
1326,ANTOINE,1_00_24,pho,NaN,NaN,a
1342,ANTOINE,1_00_24,pho,NaN,NaN,ajabə


Suppression des valeur NaN

In [7]:
pho_antoine = pho_antoine.dropna(subset=['type', 'contenu'])

Toutes les lignes du corpus strictement égales à un item

In [8]:
pho_antoine.loc[pho_antoine['contenu'] == 'a'].head()

,enfant,age,type,seconde_debut,seconde_fin,contenu
1228,ANTOINE,1_00_24,pho,NaN,NaN,a
1230,ANTOINE,1_00_24,pho,NaN,NaN,a
1326,ANTOINE,1_00_24,pho,NaN,NaN,a
1363,ANTOINE,1_00_24,pho,NaN,NaN,a
1385,ANTOINE,1_00_24,pho,NaN,NaN,a


Toutes les lignes du corpus contenant un item

In [9]:
def df_item_in_rows(item, df):
    '''
        param  : item (str)
                 df (pd.DataFrame)
        return : Un pd.DataFrame contenant les lignes du corpus qui contiennent un item
    '''
    list = []
    f=0
    for i in df['contenu']:            # On parcour la colonne contenu
        if item in i:                  # Pour chaque ligne de 'contenu'
            list.append(df.iloc[f])
        f+=1
    return pd.DataFrame(list)

In [10]:
df_item_in_rows('bɛ̃', pho_antoine).head()

,enfant,age,type,seconde_debut,seconde_fin,contenu
1556,ANTOINE,1_00_24,pho,NaN,NaN,ə bɛ̃
5858,ANTOINE,1_01_24,pho,NaN,NaN,bɛ̃
6856,ANTOINE,1_01_24,pho,NaN,NaN,bɛ̃
10170,ANTOINE,1_03_16,pho,NaN,NaN,bɛ̃
11228,ANTOINE,1_03_16,pho,NaN,NaN,bɛ̃


In [11]:
# Suppression des valeurs nan
#df_pho = pd.DataFrame(pho_antoine[{'age', 'contenu'}]).dropna()
#df_pho

Ici nous avons nos données préparées

### DataFrame selon la structure établie 

In [12]:
'''
df = pho_antoine[{'age', 'contenu'}].groupby('age') # On regroupe les phrases par l'age de l'enfant (par enregistrement)
df = pd.DataFrame(df)
len(df[1][0]['contenu']) # Pour le 1er enregistrement on a 186 phrases
df[1][0]['contenu'].str.split(' ').tolist()
df[1][1]['contenu'].values
sequences = []
for i in range(df.shape[0]):
    tup = tuple(df[1][i]['contenu'].str.split(' ').tolist())
    sequences.append(tup)
sequences[0] # On retrouve notre liste des mots par phrase du 1er enregistrement
len(sequences[0])
relim_input = itemmining.get_relim_input(sequences[6])
item_sets = itemmining.relim(relim_input, min_support=50)
item_sets
freq_seqs = seqmining.freq_seq_enum(sequences[30], 10)
for seq in freq_seqs:
    print(seq)
'''

"\ndf = pho_antoine[{'age', 'contenu'}].groupby('age') # On regroupe les phrases par l'age de l'enfant (par enregistrement)\ndf = pd.DataFrame(df)\nlen(df[1][0]['contenu']) # Pour le 1er enregistrement on a 186 phrases\ndf[1][0]['contenu'].str.split(' ').tolist()\ndf[1][1]['contenu'].values\nsequences = []\nfor i in range(df.shape[0]):\n    tup = tuple(df[1][i]['contenu'].str.split(' ').tolist())\n    sequences.append(tup)\nsequences[0] # On retrouve notre liste des mots par phrase du 1er enregistrement\nlen(sequences[0])\nrelim_input = itemmining.get_relim_input(sequences[6])\nitem_sets = itemmining.relim(relim_input, min_support=50)\nitem_sets\nfreq_seqs = seqmining.freq_seq_enum(sequences[30], 10)\nfor seq in freq_seqs:\n    print(seq)\n"

# 1. Retranscription phonétique
---

## Item : mot

In [13]:
transactions = pho_antoine['contenu'].str.split(' ').tolist()
#transactions

In [14]:
item_col = pd.DataFrame(transactions)
item_col = pd.unique(item_col.values.ravel())
item_col = pd.DataFrame(item_col)
item_col = item_col.dropna()
print(len(item_col))
item_col.head()

6089


,0
0,a
2,ahaa
3,ajabə
4,abi
5,bwaha


Notre collection d'item est de 6089 mots

### Algorithm Frequent Itemset Mining

 - Quelle valeur de support minimum ? : A partir de quelle fréquence peut-on considérer qu'un item set est fréquent ? 

In [15]:
relim_input = itemmining.get_relim_input(transactions)
item_sets = itemmining.relim(relim_input, min_support=100)

In [16]:
item_sets

{frozenset({'fɛ'}): 102,
 frozenset({'twa'}): 103,
 frozenset({'na'}): 104,
 frozenset({'χ'}): 104,
 frozenset({'u'}): 108,
 frozenset({'isi'}): 109,
 frozenset({'pø'}): 114,
 frozenset({'d'}): 116,
 frozenset({'tu'}): 118,
 frozenset({'mː'}): 121,
 frozenset({'dɑ̃'}): 124,
 frozenset({'wiʃ'}): 125,
 frozenset({'ø'}): 128,
 frozenset({'ɑ̃'}): 128,
 frozenset({'fe'}): 129,
 frozenset({'kwa'}): 129,
 frozenset({'ɛl'}): 137,
 frozenset({'ja'}): 138,
 frozenset({'t'}): 138,
 frozenset({'kɔm'}): 139,
 frozenset({'vwatyʁ'}): 139,
 frozenset({'si'}): 148,
 frozenset({'aː'}): 154,
 frozenset({'papa'}): 163,
 frozenset({'mɛ'}): 167,
 frozenset({'il'}): 173,
 frozenset({'də'}): 179,
 frozenset({'vø'}): 179,
 frozenset({'lɛ'}): 189,
 frozenset({'dø'}): 197,
 frozenset({'yn'}): 199,
 frozenset({'ɛ̃'}): 201,
 frozenset({'ɔ'}): 209,
 frozenset({'mamɑ̃'}): 211,
 frozenset({'va'}): 211,
 frozenset({'ʒə'}): 212,
 frozenset({'bɛ̃'}): 214,
 frozenset({'o'}): 220,
 frozenset({'ɔ̃'}): 225,
 frozenset({'ty'

#### Assocation rules

In [17]:
rules = assocrules.mine_assoc_rules(item_sets, min_support=2, min_confidence=0.3)
rules

[(frozenset({'ʒ'}), frozenset({'e'}), 123, 0.3867924528301887)]

Comment interpréter ces résultats ?
 - Les phrases contenant un 'ʒ' elle ont 39% de chance de contenir un 'e'
 - 

### Sequential Pattern Extraction

In [18]:
sequences = pho_antoine['contenu'].str.split(' ').tolist()
sequences

[['a'],
 ['a'],
 ['ahaa'],
 ['a'],
 ['ajabə'],
 ['abi'],
 ['a'],
 ['a'],
 ['bwaha'],
 ['a'],
 ['a'],
 ['yi'],
 ['a'],
 ['a'],
 ['ɛ'],
 ['a'],
 ['aaaaaa'],
 ['ɛdəd'],
 ['aa'],
 ['abəbə', 'X'],
 ['ə', 'bɛ̃'],
 ['hihi'],
 ['a'],
 ['wa'],
 ['datatata'],
 ['tata', 'X'],
 ['hɛ'],
 ['etɛ'],
 ['eta'],
 ['nɔp'],
 ['nei'],
 ['a'],
 ['a'],
 ['e'],
 ['m'],
 ['a'],
 ['a'],
 ['a'],
 ['Xsːi'],
 ['vwala', 'tsːi'],
 ['Xma'],
 ['məwɛ̃mɛ̃'],
 ['ɲamː'],
 ['nɲininini'],
 ['nː'],
 ['aː'],
 ['mmː'],
 ['m'],
 ['m'],
 ['aa'],
 ['az'],
 ['a'],
 ['e'],
 ['aa'],
 ['wɛ̃'],
 ['a'],
 ['m'],
 ['m'],
 ['və'],
 ['m'],
 ['ɛ̃'],
 ['ɛ̃'],
 ['hm'],
 ['a'],
 ['jɛ'],
 ['ɛ'],
 ['mm'],
 ['aaa'],
 ['ɛː'],
 ['m'],
 ['anani'],
 ['nanane'],
 ['jan'],
 ['ɲa'],
 ['abububja'],
 ['sis'],
 ['a', 'aa', 'aa'],
 ['ɛː'],
 ['ɛ'],
 ['ɑ̃'],
 ['ɑ̃', 'owa'],
 ['mja'],
 ['a'],
 ['a'],
 ['a'],
 ['m', 'ɛ', 'a'],
 ['a'],
 ['a'],
 ['m', 'm', 'a', 'm', 'm'],
 ['a'],
 ['wu'],
 ['wɛ'],
 ['χ', 'wawawəwəwə'],
 ['əw', 'bəbə'],
 ['wəbə'],
 ['b', 'bəbəbəbab

In [19]:
freq_seqs = seqmining.freq_seq_enum(sequences, 100)
for seq in freq_seqs:
    print(seq)

(('də',), 179)
(('dø',), 197)
(('aː',), 154)
(('nɑ̃',), 307)
(('na',), 104)
(('t',), 138)
(('va',), 211)
(('vwatyʁ',), 139)
(('ʒə',), 212)
(('l',), 368)
(('mɛ',), 167)
(('e',), 856)
(('ɛ',), 485)
(('ɛ̃',), 201)
(('tu',), 118)
(('ɑ̃',), 128)
(('X',), 441)
(('œ̃',), 245)
(('pa',), 700)
(('wiʃ',), 125)
(('la',), 1023)
(('se',), 290)
(('ʒ',), 318)
(('œ̃̃',), 270)
(('vø',), 179)
(('si',), 148)
(('a', 'a'), 100)
(('fɛ',), 102)
(('ɔ̃',), 225)
(('o',), 220)
(('wi',), 855)
(('papa',), 163)
(('fe',), 129)
(('lə',), 342)
(('bɛ̃',), 214)
(('d',), 116)
(('i',), 487)
(('ø',), 128)
(('pø',), 114)
(('kwa',), 129)
(('ɔ',), 209)
(('mamɑ̃',), 211)
(('dɑ̃',), 124)
(('a',), 1427)
(('χ',), 104)
(('ty',), 237)
(('twa',), 103)
(('ʒ', 'e'), 114)
(('sa',), 648)
(('ɛl',), 137)
(('nɔ̃',), 579)
(('lɛ',), 189)
(('yn',), 199)
(('u',), 108)
(('ə',), 253)
(('mwa',), 249)
(('il',), 173)
(('kɔm',), 139)
(('mː',), 121)
(('sɛ',), 573)
(('ja',), 138)
(('isi',), 109)
(('m',), 627)


In [20]:
df = pho_antoine[{'age', 'contenu'}].groupby('age') # On regroupe les phrases par l'age de l'enfant (par enregistrement)
df = pd.DataFrame(df)
print(len(df[1][0]['contenu']))
print(len(df))

186
46


In [21]:
df = pho_antoine[{'age', 'contenu'}].groupby('age') # On regroupe les phrases par l'age de l'enfant (par enregistrement)
df = pd.DataFrame(df)
len(df[1][0]['contenu']) # Pour le 1er enregistrement on a 186 phrases
df[1][0]['contenu'].str.split(' ').tolist()
df[1][1]['contenu'].values
sequences = []
for i in range(df.shape[0]):
    tup = tuple(df[1][i]['contenu'].str.split(' ').tolist())
    sequences.append(tup)

In [22]:
sequences[0] # On retrouve notre liste des mots par phrase du 1er enregistrement
len(sequences[0])
relim_input = itemmining.get_relim_input(sequences[6])

item_sets = itemmining.relim(relim_input, min_support=50)
item_sets

{}

## Item : phonème

Quels sont les phonèmes uniques présent dans le corpus ? 

# /!\ TODO /!\

In [23]:
transactions = pho_antoine['contenu'].str.split('').tolist()
pho_col = pd.DataFrame(transactions)
pho_col = pd.unique(pho_col.values.ravel())
pho_col = pd.DataFrame(pho_col)
pho_col = pho_col.dropna()
print(len(pho_col))
pho_col

57


,0
0,
1,a
3,h
4,j
5,b
6,ə
7,i
8,w
9,y
10,ɛ


In [24]:
#corpus = pho_antoine['contenu'].str.split(' ').tolist()
#transactions = []
#for list in corpus:
#    transactions.append(indexed_list(list)[0])

NameError: name 'indexed_list' is not defined

#### Récupérer les phrases qui contiennent une chaîne de caractère étrange

In [25]:
for pho in pho_col.values:
    #print(str(pho[0]))
    print(' phonème : '+str(pho[0])+' - '+ str(len(df_item_in_rows(str(pho[0]), pho_antoine))))

 phonème :  - 16112
 phonème : a - 8865
 phonème : h - 284
 phonème : j - 1643
 phonème : b - 1829
 phonème : ə - 2072
 phonème : i - 4898
 phonème : w - 3034
 phonème : y - 1531
 phonème : ɛ - 3961
 phonème : d - 2070
 phonème :   - 8167
 phonème : X - 457
 phonème : ̃ - 4658
 phonème : t - 3592
 phonème : e - 3574
 phonème : n - 2652
 phonème : ɔ - 3435
 phonème : p - 3035
 phonème : m - 3494
 phonème : s - 3439
 phonème : ː - 2039
 phonème : v - 1778
 phonème : l - 3613
 phonème : ɲ - 90
 phonème : z - 496
 phonème : u - 1929
 phonème : ɑ - 2136
 phonème : o - 1456
 phonème : χ - 107
 phonème : g - 857
 phonème : k - 2651
 phonème : ø - 997
 phonème : ʁ - 3445
 phonème : œ - 880
 phonème : f - 1166
 phonème : ʃ - 1222
 phonème : ʒ - 1039
 phonème : . - 5
 phonème : : - 4
 phonème : ɥ - 331
 phonème : r - 119
 phonème : ŋ - 11
 phonème : c - 21
 phonème : ɣ - 55
 phonème : x - 2
 phonème : ɜ - 2
 phonème : ɵ - 21
 phonème : + - 1
 phonème : ʏ - 2
 phonème : q - 2
 phonème : M - 1
 ph

### Algorithm Frequent Itemset Mining

### Sequential Pattern Extraction

# 2. Retranscription phonémique
------------------------

# 3. Phonémes indexés
---

Le but est d'indexer les phonèmes en fonction de leur placement dans un mot:
 - Le phonème est placé en début de mot
 - Le phonème est placé en milieu de mot
 - Le phonème est placé en fin de mot

In [ ]:
def indexed(x):
    """
        Index de phonème
        retourne list contenant les phonèmes indexés
        param : x (string) un mot phonétique
    """
    l = []
    # On décide de recoder les phonèmes suivant pour des raisons d'encodage
    #x=x.replace('ʁ','r') 
    x=x.replace('ɑ̃','0')
    x=x.replace('ɔ̃','1') 
    x=x.replace('ɛ̃','2') 
    x=x.replace('œ̃','3') 
    x=x.replace('tʁ','4')
    x=x.replace('kʁ','5')
    x=x.replace('dʁ','6')
    x=x.replace('gʁ','7')
    
    for i in range(0,len(x)): # On parcours chaque charactère du mot
        suff = '_d'           # On considère que le 1er charactère lu est au début du mot
        if(i>0):              # Ensuite on considère que tout les autres charactères sont en milieu de mot
            suff = '_m'
            if(i==len(x)-1):  # Sauf pour le dernier charactère lu
                suff = '_f'   
        l.append(x[i]+suff)   # On ajouter le phonème indexé 
    
    return l

In [ ]:
#test = 'az'
test = 'kʁɑ̃liʁ'
#test = 'vœgaʃ'
#test = 'e'
indexed(test)

Attention la longueur des mots n'est plus la même

In [ ]:
x = 'kʁɑ̃liʁ'
print(len(x))
print(len(indexed(x)))

# /!\ Verifier dans chaque corpus que les phonèmes que l'ont a recodé sont bien écrit de la même façon par les retranscripteurs /!\

In [ ]:
def indexed_list(list):
    '''
        param : list (list)
        return
    '''
    l = []
    for word in list:
        l.append(indexed(word))
    return l

In [ ]:
a = pho_antoine[-1:]['contenu'].str.split(' ').tolist()
a = a[0]
print(a)
print(indexed_list(a))

In [ ]:
b = ['eː','bɛ','vœga','gatœ̃','kaka','dœjøkɛka','m','taka','dɛka','gøgega','kaga','gɛda','eka','memː','kakam','kaka X','gli',
 'ta','da','lœ','gɛga','ɛ̃ɑ̃','iː','gɛga','ga','ga','gɛga','ɛɛeeiː','iii','gegɛga','gagaga','mœdada','mːmmaː','mamamamamamœkaga',
 'gaʁd','awaøː','œgaʁd','vega','ge','ga','œ̃','ɛ','mː','vøga','emɛ','myjajaja','lala','ejɛːajajajajajɛjɛːɛ',
 'ejɛajajajajajajɛɛ','nenenenenja','øgagagagaga','øga','nana','ga','a','ɔːmaː','øgaga','bølala','ala','m',
 'maːmmmœmœmː','øgalaøwa','la','X œgaga œga œga','gagaga','ma','gaga','ga a aː','a gagagaga','œ','gaga','aga',
 'gagad','bøgaga','øga','gaga','bøga','aː','gaga','gaga','ala','ga','gala','ga']

# indexed_list(b)

## Item : phonème indexé

In [ ]:
corpus = pho_antoine['contenu'].str.split(' ').tolist()
transactions = []
for list in corpus:
    transactions.append(indexed_list(list)[0])
#transactions

In [ ]:
item_col = pd.DataFrame(transactions)
item_col = pd.unique(item_col.values.ravel())
item_col = pd.DataFrame(item_col)
item_col = item_col.dropna()
print(len(item_col))
item_col.head()

Notre collection d'item est de 153 phonèmes indexés

### Algorithm Frequent Itemset Mining

In [ ]:
relim_input = itemmining.get_relim_input(transactions)
item_sets = itemmining.relim(relim_input, min_support=100)

Support minimun : 100

In [ ]:
# item_sets

#### Assocation rules

In [ ]:
rules = assocrules.mine_assoc_rules(item_sets, min_support=10, min_confidence=0.5)

    Support minimum : 10
    Confidence minimum : 0.5

In [ ]:
rules